In [1]:
!pip install ultralytics -q

In [2]:
from ultralytics import YOLO
print(YOLO('yolov8n.pt'))

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [6]:
!yolo task=detect mode=val model="./runs/detect/train12/weights/best.pt" data="C:/Users/kelka/OneDrive/Documents/Custom Object Detection/CMU_Kitchen_YOLO/dataset.yaml"

Ultralytics 8.3.96 ðŸš€ Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs
                   all        160        160   0.000511      0.123     0.0013    0.00027
             bakingpan         22         22          0          0          0          0
              colander         20         20   0.000634       0.15    0.00121    0.00017
                   cup         13         13    0.00114      0.308   0.000844     0.0001
               pitcher         23         23   0.000544      0.087   0.000343   3.43e-05
              saucepan         19         19   0.000144     0.0526   8.23e-05   4.11e-05
              scissors         20         20          0          0          0          0
                shaker         17         17    0.00123      0.235    0.00661    0.00149
               thermos         26         26   0.000398      0.154    0.00135   0.000324
Speed: 1.4ms preproc


val: Scanning C:\Users\kelka\OneDrive\Documents\Custom Object Detection\CMU_Kitchen_YOLO\labels\val.cache... 160 images, 0 backgrounds, 0 corrupt: 100%|##########| 160/160 [00:00<?, ?it/s]
val: Scanning C:\Users\kelka\OneDrive\Documents\Custom Object Detection\CMU_Kitchen_YOLO\labels\val.cache... 160 images, 0 backgrounds, 0 corrupt: 100%|##########| 160/160 [00:00<?, ?it/s]

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/10 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  10%|#         | 1/10 [00:00<00:07,  1.19it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  20%|##        | 2/10 [00:01<00:04,  1.80it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  30%|###       | 3/10 [00:01<00:03,  2.21it/s]
                 Class     Images  Instances      Box(P          R      mAP50  m

In [9]:
!yolo task=detect mode=predict model="runs/detect/train12/weights/best.pt" source="C:/Users/kelka/OneDrive/Documents/Custom Object Detection/CMU_Kitchen_YOLO/images/train/bakingpan_003.jpg"

Ultralytics 8.3.96 ðŸš€ Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 72 layers, 3,007,208 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 C:\Users\kelka\OneDrive\Documents\Custom Object Detection\CMU_Kitchen_YOLO\images\train\bakingpan_003.jpg: 480x640 1 bakingpan, 42.9ms
Speed: 1.7ms preprocess, 42.9ms inference, 115.9ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict3
ðŸ’¡ Learn more at https://docs.ultralytics.com/modes/predict


In [14]:
!pip install opencv-python
import os
import cv2
import numpy as np
from tqdm import tqdm

# Paths
dataset_path = r"C:/Users/kelka/OneDrive/Documents/Custom Object Detection/CMU_Kitchen_YOLO"
mask_dir = os.path.join(dataset_path, "masks/train")
label_dir = os.path.join(dataset_path, "labels/train")
image_dir = os.path.join(dataset_path, "images/train") #get image directory

# Ensure labels directory exists
os.makedirs(label_dir, exist_ok=True)

def mask_to_yolo(mask_path, label_path, class_id=0):
    mask = cv2.imread(mask_path, 0)  # Read in grayscale
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    h, w = mask.shape
    with open(label_path, "w") as f:
        for contour in contours:
            if len(contour) >= 3:
                normalized_contour = [(x / w, y / h) for [x, y] in contour.squeeze()]
                label_line = f"{class_id} " + " ".join(f"{x:.6f} {y:.6f}" for x, y in normalized_contour) + "\n"
                f.write(label_line)

# Process all masks
for mask_file in tqdm(os.listdir(mask_dir)):
    if mask_file.endswith("_mask.png"): # only process mask files
        mask_path = os.path.join(mask_dir, mask_file)
        image_filename = mask_file.replace("_mask.png", ".jpg") # assuming jpg, change if needed
        label_filename = mask_file.replace("_mask.png", ".txt")
        label_path = os.path.join(label_dir, label_filename)

        if os.path.exists(os.path.join(image_dir, image_filename)): #only process if the image exists
            mask_to_yolo(mask_path, label_path)
        else:
            print (f"Warning: image {image_filename} not found for mask {mask_file}")

#Cleanup extra files
for label_file in os.listdir(label_dir):
    if "_mask" in label_file:
        os.remove(os.path.join(label_dir, label_file))

100%|██████████| 200/200 [00:00<00:00, 417.14it/s]


In [3]:
!yolo task=segment mode=train model=yolov8n-seg.pt data="C:/Users/kelka/OneDrive/Documents/Custom Object Detection/CMU_Kitchen_YOLO/dataset.yaml" epochs=10 imgsz=640

Ultralytics 8.3.96 ðŸš€ Python-3.11.11 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=C:/Users/kelka/OneDrive/Documents/Custom Object Detection/CMU_Kitchen_YOLO/dataset.yaml, epochs=10, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=F


train: Scanning C:\Users\kelka\OneDrive\Documents\Custom Object Detection\CMU_Kitchen_YOLO\labels\train.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|##########| 200/200 [00:00<?, ?it/s]
train: Scanning C:\Users\kelka\OneDrive\Documents\Custom Object Detection\CMU_Kitchen_YOLO\labels\train.cache... 200 images, 0 backgrounds, 0 corrupt: 100%|##########| 200/200 [00:00<?, ?it/s]

val: Scanning C:\Users\kelka\OneDrive\Documents\Custom Object Detection\CMU_Kitchen_YOLO\labels\val.cache... 160 images, 0 backgrounds, 0 corrupt: 100%|##########| 160/160 [00:00<?, ?it/s]
val: Scanning C:\Users\kelka\OneDrive\Documents\Custom Object Detection\CMU_Kitchen_YOLO\labels\val.cache... 160 images, 0 backgrounds, 0 corrupt: 100%|##########| 160/160 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]
       1/10      4.65G      1.151      3.115      4.618       1.07         16        640:   0%|          | 0/13 [00:06<?, ?it/s]
       1/10      4.65G      1.151      3.115      4.618       1.0